### Import Library

In [1]:
import tensorflow as tf
import cv2
import numpy as np
import json
import os

%matplotlib inline

In [2]:
# Check TensorFlow Version and GPU Availability
print(tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.14.0
Num GPUs Available:  0


### Load Iris Needle Dataset Mask

In [ ]:
# Load COCO File Path
f = open('dataset/iris_neddle_raw_dataset/train_annotations.coco.json', 'r')
data = json.load(f)

In [ ]:
# Define Mask Directory
mask_dir = 'dataset/iris_needle_seg_dataset/mask/'

In [ ]:
# Processing Mask
images = data['images']
annotations = data['annotations']

for image in images:
    mask = np.zeros((image['height'], image['width']))
    for annotation in annotations:
        if annotation['image_id'] == image['id']:
            seg = annotation['segmentation']
            seg = np.array(seg).reshape((-1, 1, 2)).astype(np.int32)
            cv2.fillPoly(mask, [seg], 255)
    cv2.imwrite(mask_dir + image['file_name'], mask)
    print(image['file_name'])

### Load Cataract Dataset Mask

In [6]:
# Load COCO File Path
f = open('datasets/classification_datasets/cataract_dataset_raw/550_coco_imglab.json', 'r')
data = json.load(f)

In [7]:
# Define Mask Directory
mask_dir = 'datasets/classification_datasets/cataract_dataset_segmented/mask/'

In [8]:
# Processing Mask
images = data['images']
annotations = data['annotations']

for image in images:
    mask = np.zeros((image['height'], image['width']))
    for annotation in annotations:
        if annotation['image_id'] == image['id']:
            seg = annotation['segmentation']
            seg = np.array(seg).reshape((-1, 1, 2)).astype(np.int32)
            cv2.fillPoly(mask, [seg], 255)
    if image['file_name'].endswith('.png'):
        image['file_name'] = image['file_name'].replace('.png', '.jpg')
    cv2.imwrite(mask_dir + image['file_name'], mask)
    print(image['file_name'])

with open('550_coco_imglab_fixed.json', 'w') as json_file:
    json.dump(data, json_file)

normal_1.JPG
normal_10.JPG
normal_100.jpg
normal_101.jpg
normal_102.jpg
normal_103.jpg
normal_104.jpg
normal_105.jpg
normal_106.jpg
normal_107.jpg
normal_108.jpg
normal_109.jpg
normal_11.JPG
normal_110.jpg
normal_111.jpg
normal_112.jpg
normal_113.JPG
normal_114.JPG
normal_115.JPG
normal_116.JPG
normal_117.JPG
normal_118.JPG
normal_119.JPG
normal_12.JPG
normal_120.JPG
normal_121.JPG
normal_122.JPG
normal_123.JPG
normal_124.JPG
normal_125.JPG
normal_126.JPG
normal_127.JPG
normal_128.JPG
normal_129.JPG
normal_13.JPG
normal_130.JPG
normal_131.JPG
normal_133.JPG
normal_132.JPG
normal_134.JPG
normal_135.JPG
normal_136.JPG
normal_137.JPG
normal_138.JPG
normal_139.JPG
normal_14.JPG
normal_140.JPG
normal_141.JPG
normal_142.JPG
normal_143.JPG
normal_144.JPG
normal_145.JPG
normal_146.JPG
normal_147.JPG
normal_148.JPG
normal_149.JPG
normal_15.JPG
normal_150.JPG
normal_151.JPG
normal_152.JPG
normal_153.JPG
normal_154.JPG
normal_156.JPG
normal_155.JPG
normal_157.JPG
normal_158.JPG
normal_159.JPG
nor

### Bitwise Operation for Mask Cataract Dataset

In [9]:
image_paths = []
file_names = []
for root, dirs, files in os.walk("datasets/classification_datasets/cataract_dataset_segmented/original/"):
    for file in files:
        image_paths.append(os.path.join(root, file))
        file_names.append(file)
print(len(image_paths))
print(image_paths[1])

mask_paths = []
for root, dirs, files in os.walk("datasets/classification_datasets/cataract_dataset_segmented/mask/"):
    for file in files:
        mask_paths.append(os.path.join(root, file))
print(len(mask_paths))
print(mask_paths[1])

550
datasets/classification_datasets/cataract_dataset_segmented/original/mild_10.png
550
datasets/classification_datasets/cataract_dataset_segmented/mask/mild_10.jpg


In [10]:
for image, mask, filename in zip(image_paths, mask_paths, file_names):
    x = cv2.imread(image)
    x_mask = cv2.imread(mask, cv2.IMREAD_GRAYSCALE)
    
    result = cv2.bitwise_and(x, x, mask=x_mask)
    
    cv2.imwrite("datasets/classification_datasets/cataract_dataset_segmented/bitwise/" + filename, result)

### Segmentation for Roboflow Cataract Dataset Using Trained U-Net Model

In [17]:
# Set Parameters
SEGMENTATION_MODEL_PATH = "models/segmentation_model_1/tflite/model.tflite"
TEST_PREDICT_DIR = 'datasets/classification_datasets/roboflow_cataract_dataset_raw/train/Normal/'

In [18]:
# Load TF Lite model
interpreter_segmentation = tf.lite.Interpreter(SEGMENTATION_MODEL_PATH)

In [19]:
# Load Predict Image Path
image_path = []
file_name = []
for root, dirs, files in os.walk(TEST_PREDICT_DIR):
    for file in files:
        path_og = os.path.join(root,file)
        image_path.append(path_og)
        file_name.append(file)

In [20]:
# Allocate Tensors for Input and Output of Model
interpreter_segmentation.allocate_tensors()

In [21]:
# Get Height and Width of Input Image
_, height, width, _ = interpreter_segmentation.get_input_details()[0]['shape']

In [22]:
# Run Inference for Each Image
k = 0

for path in image_path:
    # Load Image from Path and Resize
    image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (width, height)).astype(np.float32)
    image = image / 255.0
    
    # Add Batch Dimension
    input_image = np.expand_dims(image, axis=0)
    input_image = np.expand_dims(input_image, axis=-1)
    
    # Set Input and Invoke Segmentation Model
    interpreter_segmentation.set_tensor(interpreter_segmentation.get_input_details()[0]['index'], input_image)
    interpreter_segmentation.invoke()
    
    # Get Output
    output_segmentation = np.squeeze(interpreter_segmentation.get_tensor(interpreter_segmentation.get_output_details()[0]['index']))
    output_segmentation = np.where(output_segmentation > 0.9, 1, 0).astype(np.uint8)
    
    # Multiply Image with Mask
    multipy_image = cv2.imread(path)
    multipy_image = cv2.resize(multipy_image, (width, height))
    multipy_image = cv2.bitwise_and(multipy_image, multipy_image, mask=output_segmentation)
    multipy_image = cv2.resize(multipy_image, (64, 64))
    
    # Save Image
    cv2.imwrite('datasets/classification_datasets/roboflow_cataract_segmented/normal/' + file_name[k], multipy_image)
    
    k+=1

### Return Size of Image from Cataract Dataset

In [4]:
def get_image_sizes(folder_path):
    image_sizes = []
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg')):
            img = cv2.imread(os.path.join(folder_path, filename))
            if img is not None:
                size = img.shape[:2]
                if size not in image_sizes:
                    image_sizes.append(size)
    return image_sizes

folder_path = 'datasets/segmentation_datasets/cataract_seg_dataset/mask'
print(get_image_sizes(folder_path))

[(100, 100), (85, 85), (97, 97), (93, 93), (89, 89), (69, 69), (81, 81), (73, 73), (77, 77), (53, 53), (61, 61)]


### Save Red, Green, Blue, and Gray Scale Image from Cataract Dataset

In [7]:
def save_rgb_channels(file_path):
    # Membaca gambar
    img = cv2.imread(file_path)
    gray_img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    
    # Memisahkan channel warna
    blue_channel = img[:,:,0]
    green_channel = img[:,:,1]
    red_channel = img[:,:,2]
    
    # Buat numpy array untuk menyimpan channel gambar
    blue_img = np.zeros((img.shape))
    blue_img[:,:,0] = blue_channel
    green_img = np.zeros((img.shape))
    green_img[:,:,1] = green_channel
    red_img = np.zeros((img.shape))
    red_img[:,:,2] = red_channel
    
    # Menyimpan setiap channel dan grayscale ke file
    cv2.imwrite('gray_scale.png', gray_img)
    cv2.imwrite('red_channel.png', red_img)
    cv2.imwrite('green_channel.png', green_img)
    cv2.imwrite('blue_channel.png', blue_img)

# Ganti 'path_to_your_image' dengan path file gambar Anda
save_rgb_channels('datasets/classification_datasets/roboflow_cataract_dataset_raw/train/Normal/1_JPG_jpg.rf.0afb2437ee24207ca96befdf4676819a.jpg')